In [ ]:
import IPython.display
import json
import xcube.api 

from xcube.api import read_cube


Importing lake polygons

In [ ]:
file = 'Demo/Danube.geojson'
with open(file) as f:
    geom = json.load(f)


Importing a cube 

In [ ]:
cube = read_cube('Demo/l2c-cyanoalert-olci-dan-Romania_2019-08-06_20.zarr')

---
For this demo, we are focussing on the Danube Delta in Romania

In [ ]:
IPython.display.GeoJSON(geom)

The cube contains 5 time stamps:

In [ ]:
cube.time

In [ ]:
cube

Selecting Lake Razim from polygons

In [ ]:
lake_razim = geom['features'][6]

Masking the cube to obtain only the area for the lake Razim.

In [ ]:
masked_cube = xcube.api.mask_dataset_by_geometry(cube, lake_razim, no_clip=False, save_geometry_wkt=True)


Plotting chlorophyl from the masked cube for all time stamps. 

In [ ]:
masked_cube.chl_c2rcc.plot.imshow(col='time', col_wrap=5, vmin=0, vmax=20, cmap='viridis')